## Teste 1
##### Programação grosseira em busca dos melhores dados e dos melhores modelos

In [327]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import metrics
from warnings import simplefilter

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

plt.rcParams['figure.figsize'] = (25, 8)
#sns.set_style('whitegrid')
sns.set_style('white')
sns.set_context('poster')

In [328]:
file = 'datasets/dataset_cler.csv'
file = 'C:/Users/André/AppData/Roaming/MetaQuotes/Tester/D0E8209F77C8CF37AD8BF550E51FF075/Agent-127.0.0.1-3000/MQL5/Files/dataset_cler.csv'

df = pd.read_csv(file, sep=";", encoding='utf-16')
del df['Ref']
df.shape

(1445, 104)

In [329]:
df.head(2)

,ampl_m15_5,desv_amp_m15_5,body_m15_5,body_per_m15_5,pavio_sup_m15_5,pavio_sup_per_m15_5,pavio_inf_m15_5,pavio_inf_per_m15_5,high_dist_m15_5,low_dist_m15_5,...,pavio_inf_h2_1,pavio_inf_per_h2_1,high_dist_h2_1,low_dist_h2_1,type_h2_1,atr,qt_bars,oper,take,status
0,380.0,0.85,60.0,15.79,105.0,27.63,215.0,56.58,685.0,620.0,...,215.0,47.78,615.0,620.0,up,58.21,5,Sell,-40.0,Loss
1,180.0,0.67,90.0,50.00,40.0,22.22,50.0,27.78,5.0,95.0,...,215.0,47.78,615.0,620.0,up,59.64,5,Sell,-40.0,Loss


In [330]:
df['status'].value_counts()

Loss    725
Gain    720
Name: status, dtype: int64

In [331]:
df.drop(df.loc[(df['status']!='Loss') & (df['status']!='Gain')].index.values, inplace=True)

In [332]:
df['status'].value_counts()

Loss    725
Gain    720
Name: status, dtype: int64

In [333]:
#train_features = df.columns.difference(['take', 'status', 'oper'])
X = df.drop(['take','status','oper'], axis=1) #df[train_features]
y = df['status']

x_columns = X.columns

In [334]:
index_features_category = []
for index, col in enumerate(X.columns):
    if X.dtypes[index]=='object':
        index_features_category.append(index)
index_features_category[:10]

[10, 21, 32, 43, 54, 65, 76, 87, 98]

In [335]:
encoder = LabelEncoder()
encoder.fit(['inside', 'up', 'down'])
columns_categorical = X.select_dtypes(include=['object']).columns
for col_cat in columns_categorical:
    X[col_cat] = encoder.transform(X[col_cat])

ohe = OneHotEncoder(categorical_features=index_features_category)
X = ohe.fit_transform(X).toarray()

In [336]:
def fit(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
    return y_pred 

def fitAll(X_train, X_test, y_train, y_test, algorithms):
    similaridade = []
    for item in algorithms:
        name = item[0]
        algorithm = item[1]
        
        algorithm.fit(X_train, y_train)
        y_pred = algorithm.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        #print(name, ' => ', accuracy)
        similaridade.append((accuracy, name))
        
    similaridade.sort()
    similaridade.reverse()
    
    for j in similaridade:
        print(j)

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

In [337]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

standar = StandardScaler()
standar.fit(X)
X = standar.fit_transform(X)

In [338]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=True)

In [339]:
algorithms = []

algorithms.append(['DecisionTreeClassifier', DecisionTreeClassifier(random_state=0)])
algorithms.append(['RandomForestClassifier', RandomForestClassifier(n_estimators=10, random_state=0)])
algorithms.append(['GaussianNB', GaussianNB()])
algorithms.append(['RidgeClassifier', RidgeClassifier()])
algorithms.append(['MLPClassifier', MLPClassifier(random_state=True, max_iter=1000, learning_rate_init=0.0001, hidden_layer_sizes=(89))])
algorithms.append(['GradientBoostingClassifier', GradientBoostingClassifier()])
algorithms.append(['SVC', SVC(gamma='auto')])
algorithms.append(['KNeighborsClassifier', KNeighborsClassifier()])
algorithms.append(['LogisticRegression', LogisticRegression()])

fitAll(X_train, X_test, y_train, y_test, algorithms)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(0.5506912442396313, 'RandomForestClassifier')
(0.5161290322580645, 'GradientBoostingClassifier')
(0.4930875576036866, 'KNeighborsClassifier')
(0.4930875576036866, 'GaussianNB')
(0.49078341013824883, 'DecisionTreeClassifier')
(0.4838709677419355, 'MLPClassifier')
(0.47465437788018433, 'RidgeClassifier')
(0.4700460829493088, 'LogisticRegression')
(0.46543778801843316, 'SVC')


test = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000)
y_pred = fit(X_train, X_test, y_train, y_test, test)

conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

In [340]:
y_test[300:301]

462    Loss
Name: status, dtype: object

In [341]:
X_test[300]

array([0.38006231, 0.4182058 , 0.18589744, 0.4361    , 0.23255814,
       0.16458744, 0.44715447, 0.49331902, 0.26388889, 0.05373134,
       1.        , 0.12886598, 0.02278177, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.0390625 ,
       0.19034091, 0.5       , 0.21495327, 0.17251052, 0.18589744,
       0.74542463, 0.07563025, 0.12589526, 0.10569106, 0.17533448,
       0.        , 0.06629834, 0.        , 0.12827225, 0.06242497,
       0.14102564, 0.7213    , 0.02352941, 0.0349159 , 0.12195122,
       0.26532154, 0.09166667, 0.00852273, 1.        , 0.18209877,
       0.12376934, 0.00641026, 0.0299    , 0.41818182, 0.8153426 ,
       0.17757009, 0.303868  , 0.12742382, 0.09708738, 1.        ,
       0.21850394, 0.12888199, 0.05949657, 0.20764972, 0.19117647,
       0.32222801, 0.32275132, 0.53705347, 0.09029345, 0.06012024,
       0.        , 0.49409449, 0.25868726, 0.36384439, 0.60161047,
       0.43362832, 0.41987691, 0.02686567, 0.03755556, 0.07223